In [10]:
import cv2
import numpy as np

Convert the image to grayscale

In [11]:
doc_colour = cv2.imread("img.jpg")
doc_gray = 0.21 * doc_colour[:,:,0] + 0.72 * doc_colour[:,:,1] + 0.07 * doc_colour[:,:,2] 
cv2.imwrite("img_gray.jpg", doc_gray)

True

Заполняем гистограмму

In [ ]:
hist = np.zeros(256)

min = 0
max = 0
mid = 0
sum = 0

for i in range(doc_gray.shape[0]):
    for j in range(doc_gray.shape[1]):
        sum += doc_gray[i][j]
        
        if doc_gray[i][j] > max:
            max = doc_gray[i][j]
        
        if doc_gray[i][j] < min:
            min = doc_gray[i][j]
                
mid = sum / (doc_gray.shape[0] * doc_gray.shape[1])

print(min, mid, max)
print(doc_gray.shape)

for i in range(doc_gray.shape[0]):
    for j in range(doc_gray.shape[1]):
        hist[round(doc_gray[i][j])] += 1

Otsu - более оптимальная реализация

In [ ]:
d_max = -1      # максимальное значение дисперсии
threshold = 0   # искомое пороговое значение

# вспомогательные переменные для нахождения среднего значения 
# сумма произведений интенсивности на соответствующее значение из гистограммы
tmp_hist = 0;                                   # значение для всей гистограммы
for i in range(256): tmp_hist += i * hist[i]

tmp_black = 0           # первоначальное значение для класса черных пикселей
tmp_white = tmp_hist    # первоначальное значение для класса белых пикселей

sum_black = 0                                       # первоначальное количесвто черных пикселей в классе
sum_hist = doc_gray.shape[0] * doc_gray.shape[1]    # количесвто всех пикселей

# проверяем все возможные пороговые значения в порядке возрастания (0 - черный цвет, 255 - белый)
for t in range(256):
    # находим вероятности попадания произвольного пикселя в класс черных или белых пикселей
    sum_black += hist[t]
    pr_black = sum_black / sum_hist
    pr_white = 1 - pr_black
    
    # находим средние значения интенсивности для каждого класса
    tmp_black += t * hist[t]
    tmp_white -= t * hist[t]
        
    av_black = tmp_black / pr_black
    av_white = tmp_white / pr_white
    
    # вычисляем дисперсию между классами
    d_cur = pr_black * pr_white * ((av_black - av_white) ** 2)
    
    # сравниваем ее с максимальной (если больше, то изменяем пороговое значение)
    if d_max < d_cur:
        threshold = t
        d_max = d_cur
        
print("Пороговое значение - ", threshold)

# осуществляем бинаризацию
doc_bin = doc_gray

for i in range(doc_gray.shape[0]):
    for j in range(doc_gray.shape[1]):
        if doc_gray[i][j] <= threshold:
            doc_bin[i][j] = 0
        else:
            doc_bin[i][j] = 255

cv2.imwrite("img_bin_1.jpg", doc_bin)

Otsu - more understandable implementation for familiarization with the method

In [ ]:
# проверяем все возможные пороговые значения
threshold = 0
d_max = -1

for t in range(256):
    # разбиваем пиксели на 2 класса
    black = hist[0:t] 
    white = hist[t:]
    
    # находим вероятности попадания произвольного пикселя в класс черных или белых пикселей
    prob_black = np.sum(black) / np.sum(hist)
    prob_white = 1 - prob_black
    
    # находим средние значения интенсивности для каждого класса
    average_black = 0
    average_white = 0
    
    for i in range (black.shape[0]): average_black += i * black[i]
    for i in range (white.shape[0]): average_white += (i + t) * white[i]
        
    average_black /= prob_black
    average_white /= prob_white
    
    # вычисляем дисперсию между классами
    d_cur = prob_black * prob_white * ((average_black - average_white) ** 2)
    
    # сравниваем ее с максимальной (если больше, то изменяем пороговое значение)
    if d_max < d_cur:
        threshold = t
        d_max = d_cur
        
# осуществляем бинаризацию
doc_bin = doc_gray

for i in range(doc_gray.shape[0]):
    for j in range(doc_gray.shape[1]):
        if doc_gray[i][j] <= threshold:
            doc_bin[i][j] = 0
        else:
            doc_bin[i][j] = 255

cv2.imwrite("img_bin_2.jpg", doc_bin)